In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install selenium faker

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 2.2 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 2.4 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
# --- 🔧 AUTOMATION SETUP CELL (NO BROWSER OPENING) ---

import time, random, string
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ---------- CONFIGURATION ----------
BASE_URL = "https://sourcebd-deployment.vercel.app/"
CHROMEDRIVER_PATH = r"C:\webdrivers\chromedriver.exe"
REALISTIC_SPEED = True  # Slow down typing and scrolling

# ---------- HELPER FUNCTIONS ----------

def random_email():
    """Generate random email for registration"""
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(8)) + "@testmail.com"

def human_typing(element, text, delay=0.1):
    """Types like a human for more realistic interaction"""
    for ch in text:
        element.send_keys(ch)
        time.sleep(delay)

def scroll_slowly(driver, step=400, delay=0.5):
    """Scroll down the page slowly to simulate human behavior"""
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    for y in range(0, scroll_height, step):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(delay)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

def scroll_into_view(driver, element):
    """Scrolls an element into view smoothly"""
    driver.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });", element)
    time.sleep(0.8)

def slow_click(driver, element, delay=0.5):
    """Scrolls into view and clicks slowly"""
    scroll_into_view(driver, element)
    time.sleep(delay)
    element.click()
    time.sleep(delay)

def start_browser():
    """Start Chrome only when needed"""
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    wait = WebDriverWait(driver, 20)

    print("✅ Chrome WebDriver started successfully.")
    return driver, wait

print("⚙️ Setup ready. Run start_browser() to open Chrome when needed.")


⚙️ Setup ready. Run start_browser() to open Chrome when needed.


In [ ]:
# ✅ SourceBd Full Functional UI Test (Final Version)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, string, re

CHROME_PATH = r"C:\webdrivers\chromedriver.exe"
BASE_URL = "https://sourcebd-deployment.vercel.app"

def random_email():
    return "test_" + ''.join(random.choices(string.ascii_lowercase + string.digits, k=6)) + "@gmail.com"

def scroll_into_view(driver, element):
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
    time.sleep(0.8)

def slow_scroll(driver, step=400, delay=0.8):
    height = driver.execute_script("return document.body.scrollHeight")
    for y in range(0, height, step):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(delay)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# SETUP
driver = webdriver.Chrome(service=Service(CHROME_PATH))
driver.set_window_size(1280, 900)
wait = WebDriverWait(driver, 20)
driver.get(BASE_URL)
print("🚀 Test started")
time.sleep(3)
slow_scroll(driver)

# REGISTER NEW USER
email = random_email()
print(f"📝 Registering new user: {email}")
driver.get(f"{BASE_URL}/register")
time.sleep(2)
driver.find_element(By.ID, "firstName").send_keys("Rehan")
driver.find_element(By.ID, "lastName").send_keys("Tester")
driver.find_element(By.ID, "email").send_keys(email)
driver.find_element(By.ID, "phone").send_keys("01700000000")
driver.find_element(By.ID, "country").send_keys("Bangladesh")
driver.find_element(By.ID, "password").send_keys("87654321")
driver.find_element(By.ID, "confirmPassword").send_keys("87654321")
create_btn = driver.find_element(By.XPATH, "//button[contains(., 'Create Account')]")
scroll_into_view(driver, create_btn)
create_btn.click()
print("✅ Registration submitted")
time.sleep(6)

# LOGOUT (from dropdown)
print("🔓 Logging out after auto-login...")
profile_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".profile-btn")))
scroll_into_view(driver, profile_btn)
profile_btn.click()
time.sleep(1)
logout_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Logout')]")))
scroll_into_view(driver, logout_btn)
logout_btn.click()
time.sleep(3)

# LOGIN AGAIN
driver.get(f"{BASE_URL}/login")
print("🔐 Logging in again...")
driver.find_element(By.ID, "email").send_keys(email)
driver.find_element(By.ID, "password").send_keys("87654321")
signin_btn = driver.find_element(By.XPATH, "//button[contains(., 'Sign In')]")
scroll_into_view(driver, signin_btn)
signin_btn.click()
time.sleep(5)
print("✅ Logged in successfully")

# --- HOMEPAGE SCROLL ---
print("🏠 Scrolling through homepage...")
driver.get(BASE_URL)
time.sleep(2)
slow_scroll(driver)

# --- HEADER LINKS ---
for label in ["About Us", "Help", "Contact"]:
    link = wait.until(EC.element_to_be_clickable((By.XPATH, f"//a[contains(., '{label}')]")))
    scroll_into_view(driver, link)
    link.click()
    print(f"✅ Navigated to {label}")
    time.sleep(2)
    slow_scroll(driver)
    time.sleep(1)

# --- CONTACT FORM ---
print("📩 Filling contact form...")
driver.get(f"{BASE_URL}/contact")
time.sleep(2)
slow_scroll(driver)
form_fields = {
    "fullName": "Rehan Test",
    "email": email,
    "companyName": "Test Company Ltd.",
    "phoneNumber": "01700000000",
    "message": "Testing automated contact form submission."
}
for name, value in form_fields.items():
    field = driver.find_element(By.NAME, name)
    scroll_into_view(driver, field)
    field.clear()
    field.send_keys(value)
Select(driver.find_element(By.NAME, "subject")).select_by_visible_text("General Inquiry")
submit_btn = driver.find_element(By.XPATH, "//button[contains(., 'Send Message')]")
scroll_into_view(driver, submit_btn)
submit_btn.click()
print("✅ Contact form submitted successfully")
time.sleep(4)

# --- PRODUCT PAGE ---
driver.get(BASE_URL)
time.sleep(3)
print("🛍️ Opening first product...")
first_product = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".product-card a")))
scroll_into_view(driver, first_product)
driver.execute_script("arguments[0].click();", first_product)
time.sleep(4)
slow_scroll(driver)

# --- EXTRACT MINIMUM QUANTITY ---
min_texts = driver.find_elements(By.XPATH, "//*[contains(text(), 'Minimum:')]")
min_qty = 100
if min_texts:
    match = re.search(r"Minimum:\s*(\d+)", min_texts[0].text)
    if match:
        min_qty = int(match.group(1))
print(f"📦 Minimum quantity extracted: {min_qty}")

# --- WISHLIST ADD ---
try:
    print("❤️ Adding product to wishlist...")
    wishlist_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Save')]")))
    scroll_into_view(driver, wishlist_btn)
    wishlist_btn.click()
    print("✅ Product added to wishlist")
    time.sleep(2)
except Exception as e:
    print("⚠️ Wishlist add failed:", e)

# --- ADD TO CART ---
try:
    print("🛒 Adding product to cart...")
    add_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Add to Cart')]")))
    scroll_into_view(driver, add_btn)
    add_btn.click()
    print("✅ Product added to cart")
    time.sleep(2)
except Exception as e:
    print("⚠️ Add to cart failed:", e)



# --- WISHLIST PAGE ---
print("🧡 Checking wishlist page...")
profile_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".profile-btn")))
scroll_into_view(driver, profile_btn)
profile_btn.click()
time.sleep(1)
wish_link = driver.find_element(By.XPATH, "//a[contains(., 'Wishlist')]")
scroll_into_view(driver, wish_link)
wish_link.click()
time.sleep(3)
slow_scroll(driver)
print("✅ Wishlist page verified")

# --- CART PAGE ---
cart_link = driver.find_element(By.CSS_SELECTOR, ".cart-link")
scroll_into_view(driver, cart_link)
cart_link.click()
time.sleep(3)
slow_scroll(driver)
print("✅ Cart opened")

# --- CART PAGE: Proceed to Checkout ---
try:
    print("🛒 Checking cart page and proceeding to checkout...")
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".cart-page")))
    scroll_slowly(driver)
    time.sleep(2)

    # --- Find Proceed to Checkout link ---
    print("🔎 Looking for Proceed to Checkout link...")
    proceed_btn = None
    try:
        proceed_btn = wait.until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//a[contains(., 'Proceed to Checkout') or contains(@href, '/checkout')]"
            ))
        )
    except:
        try:
            proceed_btn = wait.until(
                EC.element_to_be_clickable((
                    By.CSS_SELECTOR,
                    "a.btn.btn-primary.btn-full[href*='/checkout']"
                ))
            )
        except:
            pass

    if not proceed_btn:
        raise Exception("Proceed to Checkout link not found.")

    scroll_into_view(driver, proceed_btn)
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", proceed_btn)
    time.sleep(1.5)
    proceed_btn.click()
    print("✅ Clicked 'Proceed to Checkout' link successfully.")

    # --- Wait for checkout page ---
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".checkout-page")))
    time.sleep(2)

    print("🧾 Filling checkout form...")
    fields = {
        "firstName": "Test",
        "lastName": "User",
        "email": "test_checkout@example.com",
        "phone": "01712345678",
        "address": "123 Test Street",
        "city": "Dhaka",
        "state": "Dhaka Division",
        "zipCode": "1207",
        "country": "Bangladesh"
    }

    for name, value in fields.items():
        try:
            selector = f"input[name='{name}'], select[name='{name}']"
            input_el = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
            scroll_into_view(driver, input_el)
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", input_el)
            time.sleep(0.6)
            input_el.clear()
            input_el.send_keys(value)
            print(f"✏️ Filled {name}: {value}")
        except Exception as field_error:
            print(f"⚠️ Could not fill field {name}: {field_error}")

    # --- Click Place Order button ---
    try:
        place_order_btn = wait.until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//button[contains(., 'Place Order') or contains(., 'Confirm Order') or contains(., 'Submit Order')]"
            ))
        )
        scroll_into_view(driver, place_order_btn)
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", place_order_btn)
        time.sleep(1)
        place_order_btn.click()
        print("✅ Clicked 'Place Order' button successfully.")
    except Exception as place_err:
        print("⚠️ Could not click 'Place Order' button:", place_err)

    time.sleep(3)

except Exception as e:
    print("⚠️ Proceed to checkout failed:", e)



# --- MY PROFILE PAGE: Edit and Save ---
try:
    print("👤 Editing profile information...")
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".profile-page")))
    scroll_slowly(driver)
    time.sleep(1)

    # Try clicking "Edit" button first
    try:
        edit_btn = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Edit') or contains(., 'Edit Profile')]"))
        )
        scroll_into_view(driver, edit_btn)
        edit_btn.click()
        print("🖊️ Edit button clicked.")
    except:
        print("⚠️ Could not find edit button, proceeding to fields directly...")

    time.sleep(2)

    # Fill editable fields
    new_values = {
        "firstName": "Updated",
        "lastName": "Tester",
        "phone": "01812345678",
        "company": "AutoTest Ltd",
        "address": "Updated Address 2025"
    }

    for name, value in new_values.items():
        try:
            input_field = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, f"input[name='{name}']"))
            )
            scroll_into_view(driver, input_field)
            input_field.clear()
            input_field.send_keys(value)
            time.sleep(0.6)
        except:
            pass

    # Click Save button
    try:
        save_btn = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Save') or contains(., 'Update Profile')]"))
        )
        scroll_into_view(driver, save_btn)
        time.sleep(1)
        save_btn.click()
        print("✅ Profile information updated successfully.")
    except:
        print("⚠️ Save button not found or not clickable.")

    time.sleep(2.5)

except Exception as e:
    print("⚠️ Edit profile failed:", e)


# --- MY ORDERS ---
print("📦 Checking My Orders page...")
profile_btn = driver.find_element(By.CSS_SELECTOR, ".profile-btn")
scroll_into_view(driver, profile_btn)
profile_btn.click()
time.sleep(1)
orders_link = driver.find_element(By.XPATH, "//a[contains(., 'My Orders')]")
scroll_into_view(driver, orders_link)
orders_link.click()
time.sleep(3)
slow_scroll(driver)
print("✅ My Orders page verified")

# DONE
print("🎉 All tests completed successfully!")
time.sleep(3)
driver.quit()
print("✅ Browser closed.")


🚀 Test started
📝 Registering new user: test_l4sjl6@gmail.com
✅ Registration submitted
🔓 Logging out after auto-login...
🔐 Logging in again...
✅ Logged in successfully
🏠 Scrolling through homepage...
✅ Navigated to About Us
✅ Navigated to Help
✅ Navigated to Contact
📩 Filling contact form...
✅ Contact form submitted successfully
🛍️ Opening first product...
📦 Minimum quantity extracted: 100
❤️ Adding product to wishlist...
✅ Product added to wishlist
🛒 Adding product to cart...
✅ Product added to cart
🧡 Checking wishlist page...
✅ Wishlist page verified
✅ Cart opened
🛒 Checking cart page and proceeding to checkout...
🔎 Looking for Proceed to Checkout link or button...
✅ Clicked 'Proceed to Checkout' link successfully.
🧾 Filling checkout form...
✏️ Filled firstName: Test
✏️ Filled lastName: User
✏️ Filled email: test_checkout@example.com
✏️ Filled phone: 01712345678
✏️ Filled address: 123 Test Street
✏️ Filled city: Dhaka
✏️ Filled state: Dhaka Division
⚠️ Could not fill field postalCode:

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff6390ae9e5+80021]
	GetHandleVerifier [0x0x7ff6390aea40+80112]
	(No symbol) [0x0x7ff638e30425]
	(No symbol) [0x0x7ff638e7864f]
	(No symbol) [0x0x7ff638eb10f2]
	(No symbol) [0x0x7ff638eab89f]
	(No symbol) [0x0x7ff638eaa94a]
	(No symbol) [0x0x7ff638dfa765]
	GetHandleVerifier [0x0x7ff63936dd8d+2960445]
	GetHandleVerifier [0x0x7ff63936804a+2936570]
	GetHandleVerifier [0x0x7ff639388a87+3070263]
	GetHandleVerifier [0x0x7ff6390c84ce+185214]
	GetHandleVerifier [0x0x7ff6390cff1f+216527]
	(No symbol) [0x0x7ff638df970e]
	GetHandleVerifier [0x0x7ff6394a6b78+4241960]
	BaseThreadInitThunk [0x0x7ffc473ce8d7+23]
	RtlUserThreadStart [0x0x7ffc48e8c53c+44]
